# Video Test ( Demo)

In [ ]:
from PIL import Image
from keras.applications.vgg16 import preprocess_input
import base64
from io import BytesIO
import json
import random
import cv2
from keras.models import load_model
import numpy as np
from datetime import datetime

from keras.preprocessing import image
model = load_model('CNN_inception_transfer.h5')   # change model


model_dict = {
    0: 'Adriana Lima',
    1: 'Alexandra Daddario',
    2: 'Alvaro Morte',
    3: 'Anne Hathaway',
    4: 'Anthony Mackie',
    5: 'Ben Affleck',
    6: 'Bill Gates',
    7: 'amber heard',
    8: 'barack obama',
    }



# Loading the cascades
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_extractor(img):
    faces = face_cascade.detectMultiScale(img, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

# Doing some Face Recognition with the webcam
# video_capture = cv2.VideoCapture(0)
video_capture = cv2.VideoCapture("Demo5.mp4")

while True:
    _, frame = video_capture.read()    
    face=face_extractor(frame)
    if type(face) is np.ndarray:
        face = cv2.resize(face, (180, 180))
        im = Image.fromarray(face, 'RGB')
           #Resizing into 128x128 because we trained the model with this image size.
        img_array = np.array(im)
                    #Our keras model used a 4D tensor, (images x height x width x channel)
                    #So changing dimension 128x128x3 into 1x128x128x3 
        img_array = np.expand_dims(img_array, axis=0)
        pred = model.predict(img_array)[0]
        maxindex = int(np.argmax(pred))
        print(maxindex)
                     
        name="None matching"
        now = datetime.now()
        date = now.strftime("%d/%m/%Y %H:%M:%S")
        print(pred[maxindex])
        if pred[maxindex] > .4:
            name = model_dict[maxindex]
#         if(pred[0][3]>0.3):
        my_text = f"{name} - {date}"
            
        cv2.putText(frame,my_text, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
    else:
        cv2.putText(frame,"No face found", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
    try:    
        cv2.imshow('Video', frame)
    except:
        print("Video has ended.")
        video_capture.release()
        cv2.destroyAllWindows()
        break
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()